In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Логистическая регрессия
Логистическая регрессия используется для задач классификации, 
при этом вычисляется вероятность принадлежности  
события к определенному классу.  
  
Задача алгоритма логистической регрессии - найти  
подходящие коэффициенты <code>w</code> при признаках <code>x</code>:  

<code>w<sub>0</sub> + w<sub>1</sub>x<sub>1</sub> + w<sub>2</sub>x<sub>2</sub> + ... +  + w<sub>m</sub>x<sub>m</sub></code>  
  
Величина z помещается в сигмоидную функцию для вычисления вероятности:  
  
<code>f(x) = 1 / (1 + e<sup>-z</sup>)</code>  
  
Значение <code>f(x)</code> будет расположено на отрезке [0, 1].  
<code>f(x)</code> - вероятность отнесения события к классу 1  
<code>1 - f(x)</code> - вероятность отнесения события к классу 0  
  
**Log loss**  
Алгоритм логистической регрессии минимизирует величину logloss:  
<code>logloss = -y * ln(p) - (1 - y) * ln(1 - p)</code>  
где <code>y</code> - истинное значение (0 или 1)  
p - вычисленная алгоритмом вероятность того, что событие принадлежит классу 1  (***f(x)***)  

### Titanic
Для изучения классификации мы будем использовать  
датасет с информацией о пассажирах Титаник с сайта Kaggle.com

In [3]:
data = pd.read_csv('input/train.csv')

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Установим в качестве индекса PassengerId - так нам будет проще отслеживать, с информацией о каком пассажире мы работаем

In [10]:
data = data.set_index('PassengerId')

In [11]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
data.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Наша задача - определить, выжил ли человек при крушении Титаника  
В поле Survived выжившие пассажиры обозначены единицей, а утонувшие - нулем  

In [13]:
target = 'Survived'
y = data[target]
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64

In [14]:
X = data.drop(target, axis = 1)

In [15]:
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Изуление качества данных и их очистка

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


В поле Cabin слишком много пропущенных значений,  
в этом уроке мы не будем го использовать.  
Стобцы Name и Ticket также в этот раз не рассматриваем.
Удалим эти поля:

In [17]:
X = X.drop(['Cabin', 'Name', 'Ticket'], axis = 1)

В поле возраст 20% значений не заполнено. Заменим пропущенные значения на значения среднего возраста.

In [18]:
mean_age = X['Age'].mean()
mean_age

29.69911764705882

In [19]:
X['Age'] = X['Age'].fillna(mean_age)
X['Age'].unique()

array([22.        , 38.        , 26.        , 35.        , 29.69911765,
       54.        ,  2.        , 27.        , 14.        ,  4.        ,
       58.        , 20.        , 39.        , 55.        , 31.        ,
       34.        , 15.        , 28.        ,  8.        , 19.        ,
       40.        , 66.        , 42.        , 21.        , 18.        ,
        3.        ,  7.        , 49.        , 29.        , 65.        ,
       28.5       ,  5.        , 11.        , 45.        , 17.        ,
       32.        , 16.        , 25.        ,  0.83      , 30.        ,
       33.        , 23.        , 24.        , 46.        , 59.        ,
       71.        , 37.        , 47.        , 14.5       , 70.5       ,
       32.5       , 12.        ,  9.        , 36.5       , 51.        ,
       55.5       , 40.5       , 44.        ,  1.        , 61.        ,
       56.        , 50.        , 36.        , 45.5       , 20.5       ,
       62.        , 41.        , 52.        , 63.        , 23.5 

В поле Embarked, которое являтся текстовым, не хватает двух значений,  
заполним пропущеные значения 

In [20]:
X['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [22]:
embarked_mode = X['Embarked'].mode()[0]
embarked_mode

'S'

In [24]:
X['Embarked'] = X['Embarked'].fillna(embarked_mode)
X['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [25]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 55.7+ KB


### Работа с категориальными признаками
Признак пола пассажира (Sex) - категориальная переменная. Так как в ней два класса,  
то мы можем представить одной колонкой со  
значениями 0 и 1:

In [26]:
X['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [27]:
X['Sex'] = (X['Sex'] == 'female').astype(int)
X['Sex'].value_counts()

0    577
1    314
Name: Sex, dtype: int64

Еще одна категориальная переменная - Embarked.
Преобразуем ее значения в dummy-переменные.

In [28]:
X = pd.get_dummies(X)
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,0,22.0,1,0,7.2500,0,0,1
2,1,1,38.0,1,0,71.2833,1,0,0
3,3,1,26.0,0,0,7.9250,0,0,1
4,1,1,35.0,1,0,53.1000,0,0,1
5,3,0,35.0,0,0,8.0500,0,0,1


In [29]:
X.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    uint8  
 7   Embarked_Q  891 non-null    uint8  
 8   Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(4), uint8(3)
memory usage: 51.3 KB


In [30]:
X['Pclass'].unique()

array([3, 1, 2])

In [31]:
X['Pclass'] = X['Pclass'].astype('category')

In [32]:
X = pd.get_dummies(X)
X.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3'],
      dtype='object')

In [33]:
X.head(10)

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
1,0,22.000000,1,0,7.2500,0,0,1,0,0,1
2,1,38.000000,1,0,71.2833,1,0,0,1,0,0
3,1,26.000000,0,0,7.9250,0,0,1,0,0,1
4,1,35.000000,1,0,53.1000,0,0,1,1,0,0
5,0,35.000000,0,0,8.0500,0,0,1,0,0,1
6,0,29.699118,0,0,8.4583,0,1,0,0,0,1
7,0,54.000000,0,0,51.8625,0,0,1,1,0,0
8,0,2.000000,3,1,21.0750,0,0,1,0,0,1
9,1,27.000000,0,2,11.1333,0,0,1,0,0,1


### Цель разбиения данных на тренировочный, валидационный и тестовый датасеты
В этот раз будет более работать с разделением данных для модели более тщательно,  
мы разобьем данные из файла train.csv на две части:  
тренировачный набор данных (признаки в X_train, целевая переменная в y_train)  
и валидационные (соответственно X_valid, y_valid).  
На тренировочном датасете мы будем строить модель, а на валидационном - проверять ее качество.  
После проверки на валидационном сете можно будет окончательно проверить модель на тестовом сете  
(содержится в файле test.csv по вышеуказанной ссылке -  
фактическое значение целевой переменной в тестовом наборе отсутствует,  
рекомендуем самостоятельно проверить точность данных при отправке решенияна странице соревнования).


In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, random_state = 42)

### Построение модели
В начале работы над любой задачей рекомендуется сначала сделать простую модель,  
не затрачивая большого количества времени.  
Замем можно будет оценивать новые модели, сравнивая их качество  
с качеством первональной (базовой) модели.  
  


Сначала посмотрим на информацию о признаках:

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 299 to 103
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sex         668 non-null    int64  
 1   Age         668 non-null    float64
 2   SibSp       668 non-null    int64  
 3   Parch       668 non-null    int64  
 4   Fare        668 non-null    float64
 5   Embarked_C  668 non-null    uint8  
 6   Embarked_Q  668 non-null    uint8  
 7   Embarked_S  668 non-null    uint8  
 8   Pclass_1    668 non-null    uint8  
 9   Pclass_2    668 non-null    uint8  
 10  Pclass_3    668 non-null    uint8  
dtypes: float64(2), int64(3), uint8(6)
memory usage: 35.2 KB


Все признаки - числовые, поэтому мы можем построить модель, используя все признаки.

In [37]:
lr = LogisticRegression()

In [38]:
lr.fit(X_train, y_train)

LogisticRegression()

In [39]:
y_pred = lr.predict(X_valid)
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0])

Метрика Accuracy (доля правильных ответов)

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
accuracy_score(y_valid, y_pred)

0.8026905829596412

### Вычисление вероятности событий

In [42]:
y_proba = lr.predict_proba(X_valid)

In [44]:
# Вероятности событий 0 и 1 для каждого пассажира
y_proba

array([[0.88803847, 0.11196153],
       [0.7306994 , 0.2693006 ],
       [0.8701565 , 0.1298435 ],
       [0.08991937, 0.91008063],
       [0.25062013, 0.74937987],
       [0.07826308, 0.92173692],
       [0.33446955, 0.66553045],
       [0.90522439, 0.09477561],
       [0.2454247 , 0.7545753 ],
       [0.10340606, 0.89659394],
       [0.69433675, 0.30566325],
       [0.93487212, 0.06512788],
       [0.6248471 , 0.3751529 ],
       [0.84761359, 0.15238641],
       [0.75859309, 0.24140691],
       [0.07898047, 0.92101953],
       [0.72748705, 0.27251295],
       [0.33438488, 0.66561512],
       [0.70247546, 0.29752454],
       [0.70651164, 0.29348836],
       [0.88386344, 0.11613656],
       [0.64286508, 0.35713492],
       [0.39821921, 0.60178079],
       [0.86944939, 0.13055061],
       [0.89833287, 0.10166713],
       [0.92821101, 0.07178899],
       [0.56597441, 0.43402559],
       [0.72832999, 0.27167001],
       [0.91442869, 0.08557131],
       [0.42638415, 0.57361585],
       [0.

Нас интересует вероятность события 1 (правый столбец) -  
при получении ответа значения из него округляется до 0 или до 1  


***Сохранение данных***  
  
Сохраним обработанные данные, они пригодятся для следующих уроков

In [45]:
X_train.to_pickle('X_train.pkl')
y_train.to_pickle('y_train.pkl')

X_valid.to_pickle('X_valid.pkl')
y_valid.to_pickle('y_valid.pkl')